In [12]:
import numpy as np
import math
from scipy.stats import t
import matplotlib.pyplot as plt

# Parameters 
m = 10
mst = 8
mtbc = 1

n_sim = 10
block_fractions = np.zeros(n_sim)
block_Z = np.zeros(n_sim)

for j in range(n_sim):
    n_custom = 10000
    n = 0

    interval_between_arrivals = np.random.exponential(scale = mtbc, size = n_custom)
    arrival_times = np.cumsum(interval_between_arrivals)

    service_in_use_times = np.zeros(m)

    n_cust_blocked = 0


    while(n < n_custom):

        available_services = np.where(arrival_times[n] >= service_in_use_times)[0]
        if (len(available_services)>0):
            i = available_services[0]
            service_time = np.random.exponential(scale=mst)
            service_in_use_times[i] = arrival_times[n] + service_time
        else:
            n_cust_blocked += 1
        n += 1

    fraction = n_cust_blocked / n_custom  * 100
    print(f'fraction of blocks: {fraction}%')
    block_fractions[j] = fraction

# Exact solution
A = mst * mtbc
m_vect = np.arange(0,m+1)
denominator = np.sum(A**m_vect / [math.factorial(k) for k in m_vect])
B = A**m/math.factorial(m) / denominator

print(f'exact solution: {B*100}%')

# Confidence interval

theta_hat = np.sum(block_fractions)/n_sim
#print(theta_hat)
sigma_2 = (np.sum(block_fractions ** 2) - n_sim * theta_hat**2) / (n_sim -1)
print(sigma_2)  

dof = n_sim -1
alpha = 0.95
t_quant = t.ppf(1-alpha/2,dof)
CI = [theta_hat - np.sqrt(sigma_2)/np.sqrt(n)*t_quant, theta_hat + np.sqrt(sigma_2)/np.sqrt(n)*t_quant]

print(f'Confidence interval: {CI}')

U = np.random.uniform(0,1,size = n_sim)
c = - np.cov(block_fractions, U)[0, 1] / np.var(U)
Z = block_fractions + c*(U-0.5)
Z_cv = np.sum(Z)/n_sim
Z_var = (np.sum(Z ** 2) - n_sim * Z_cv**2) / (n_sim -1)

print(f'Control variate estimate: {Z_cv}')

print(f'Original variance: {sigma_2}, Control variate variance {Z_var}')

fraction of blocks: 10.83%
fraction of blocks: 11.23%
fraction of blocks: 12.04%
fraction of blocks: 11.44%
fraction of blocks: 12.29%
fraction of blocks: 13.459999999999999%
fraction of blocks: 12.68%
fraction of blocks: 11.52%
fraction of blocks: 12.35%
fraction of blocks: 11.42%
exact solution: 12.16610642529515%
0.6210711111111019
Confidence interval: [11.925491870893383, 11.926508129106617]
Control variate estimate: 11.982350446171614
Original variance: 0.6210711111111019, Control variate variance 0.5968861341069037
